# 归档页面

当用户点击归档下的某个日期或者分类下的某个分类时，跳转到文章列表页面，显示该日期或者分类下的全部文章。

主页视图函数中我们通过 Post.objects.all() 获取全部文章，而在我们的归档和分类视图中，我们不再使用 all 方法获取全部文章，而是使用 filter 来根据条件过滤。先来看归档视图：

In [ ]:
# blog/views.py

def archives(request, year, month):
    post_list = Post.objects.filter(created_time__year=year,
                                    created_time__month=month
                                    ).order_by('-created_time')
    return render(request, 'blog/index.html', context={'post_list': post_list})

根据 created_time 的 year 和 month 属性过滤，筛选出文章发表在对应的 year 年和 month 月的文章。

created_time 是 Python 的 date 对象，其有一个 year 和 month 属性

Python 中类实例调用属性的方法通常是 created_time.year，但是由于这里作为函数的参数列表，所以 Django 要求我们把点替换成了两个下划线，即 created_time__year。

**配置好 URL：**

In [ ]:
# blog/urls.py

from django.conf.urls import url

from . import views

app_name = 'blog'
urlpatterns = [
    url(r'^$', views.index, name='index'),
    url(r'^post/(?P<pk>[0-9]+)/$', views.detail, name='detail'),
    + url(r'^archives/(?P<year>[0-9]{4})/(?P<month>[0-9]{1,2})/$', views.archives, name='archives'),
]

例如如果用户想查看 2017 年 3 月下的全部文章，他访问 /archives/2017/3/，那么 archives 视图函数的实际调用为：archives(request, year=2017, month=3)。

In [ ]:
# templates/base.html

{% for date in date_list %}
<li>
  <a href="{% url 'blog:archives' date.year date.month %}">
    {{ date.year }} 年 {{ date.month }} 月
  </a>
</li>
{% endfor %}

**{% url %}** 这个模板标签的作用是**解析视图函数** blog:archives 对应的 URL 模式，并把 URL 模式中的年和月替换成 date.year，date.month 的值。

我们把超链接的 href 属性设置为 /archives/{{ date.year }}/{{ date.month }}/ 同样可以达到目的，但是这种写法是硬编码的。虽然现在 blog:archives 视图函数对应的 URL 模式是这种形式，但是如果哪天这个模式改变了呢？如果使用了硬编码的写法，那你需要把每一处 /archives/{{ date.year }}/{{ date.month }}/ 修改为新的模式。但如果使用了 {% url %} 模板标签，则不用做任何修改。

# 分类页面

In [ ]:
# blog/views.py

import markdown

from django.shortcuts import render, get_object_or_404

# 引入 Category 类
from .models import Post, Category

def category(request, pk):
    # 记得在开始部分导入 Category 类
    cate = get_object_or_404(Category, pk=pk)
    post_list = Post.objects.filter(category=cate).order_by('-created_time')
    return render(request, 'blog/index.html', context={'post_list': post_list})

get_object_or_404 函数和 detail 视图中一样，其作用是如果用户访问的分类不存在，则返回一个 404 错误页面以提示用户访问的资源不存在。

**URL 配置如下**

In [ ]:
# blog/urls.py

from django.conf.urls import url

from . import views

app_name = 'blog'
urlpatterns = [
    url(r'^$', views.index, name='index'),
    url(r'^post/(?P<pk>[0-9]+)/$', views.detail, name='detail'),
    url(r'^archives/(?P<year>[0-9]{4})/(?P<month>[0-9]{1,2})/$', views.archives, name='archives'),
    + url(r'^category/(?P<pk>[0-9]+)/$', views.category, name='category'),
]

**修改相应模板：**

```HTML
templates/base.html

{% for category in category_list %}
<li>
  <a href="{% url 'blog:category' category.pk %}">{{ category.name }}</a>
</li>
{% endfor %}
```